# Can Large Language Models Learn Statistics? A RAG Experiment
**Author:** Shourya Marwaha  
**Context:** MSc Dissertation Project

## 📄 Abstract
This project investigates whether Large Language Models (LLMs) genuinely "learn" and integrate new statistical concepts when provided with external textbooks via **Retrieval Augmented Generation (RAG)**. 

Using a **$2\times3$ factorial design**, I evaluated three distinct model architectures (Llama 3.2, DeepSeek-R1, Qwen2-Math) across 50 graduate-level statistics questions. The study utilizes a custom **"LLM-as-a-Judge"** evaluation framework to assess Correctness, Explanation Quality, and Understanding.

## 🛠️ Technical Methodology
* **RAG Pipeline:** ChromaDB (Vector Store) + Cross-Encoder Reranking (`ms-marco-MiniLM-L-6-v2`)
* **Embeddings:** `all-mpnet-base-v2`
* **Evaluation:** Automated rubric grading using a Judge LLM validated against human consensus.

In [ ]:
# --- Imports & Configuration ---
import os
import re
import json
import time
import math
import warnings
import multiprocessing as mp
from pathlib import Path
from typing import List, Dict, Any, Tuple, Set
from collections import Counter
from functools import lru_cache
import glob

# Data Science
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# NLP & AI
import nltk
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
import ollama
from openai import OpenAI
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import chromadb

# LangChain & LlamaIndex
from langchain.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# PDF Processing
from pdf2image import convert_from_path
from pytesseract import pytesseract
import requests

from dotenv import load_dotenv

# --- Setup ---
load_dotenv()
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PATH"] += os.pathsep + '/opt/homebrew/bin'
mp.set_start_method('spawn', force=True)

# Download NLTK data
nltk_packages = ["wordnet", "omw-1.4", "stopwords", "punkt"]
for pkg in nltk_packages:
    nltk.download(pkg, quiet=True)

# Initialize Clients
ollama_client = ollama.Client()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

# Data Paths
dataset_books = os.getenv("DATASET_BOOKS")
dataset_chunk = os.getenv("DATASET_CHUNK") 
dataset_main = os.getenv("DATASET_MAIN")

# Initialize Models
# Using all-mpnet-base-v2 for robust semantic embeddings
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
# Using CrossEncoder for high-precision reranking
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Database Setup
client = chromadb.PersistentClient(path="chroma_db")
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="stats_books")

print("✅ Environment Setup Complete.")

In [ ]:
# --- Document Processing Pipeline ---

def load_documents(data_path):
    """Loads PDF documents from the specified directory."""
    # Find all PDF files
    pdf_files = glob.glob(os.path.join(data_path, "**/*.pdf"), recursive=True)
    
    documents = []
    
    for file_path in pdf_files:
        file_name = Path(file_path).name
        print(f"   📖 Loading: {file_name}")
    
        loader = UnstructuredPDFLoader(file_path)
        file_docs = loader.load()
            
        for doc in file_docs:
            doc.metadata['source_file'] = file_name
            documents.extend(file_docs)
            print(f"   ✅ Loaded {len(file_docs)} chunks from {file_name}")
    
    print(f"\n📊 Total: {len(documents)} chunks from {len(pdf_files)} files")
    return documents

def split_text(documents, chunk_size, overlap):
    """Splits documents into smaller chunks for vector storage."""
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap,
    length_function=len,
    separators=[ "\n\n## ",  "\n\n# ",  "\n\nExample",  "\n\n",  "\n",  ". ", " "  ])                           

    chunks = text_splitter.split_documents(documents)

    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks
    
# Load the data
# documents = load_documents(dataset_books) # Uncomment to run

In [ ]:
# --- Query Transformations (Multi-Query Expansion) ---

_STOP = set(stopwords.words("english"))
_LEM  = WordNetLemmatizer()

def _normalize_tokens(text: str) -> List[str]:
    return re.findall(r"[A-Za-z0-9]+", text.lower())

def _lemmatize(tokens: List[str]) -> List[str]:
    return [_LEM.lemmatize(t) for t in tokens]

def condense_query(query: str) -> str:
    """Simplifies the query to keywords."""
    toks = _normalize_tokens(query)
    toks = [t for t in toks if t not in _STOP]
    toks = _lemmatize(toks)
    return " ".join(toks)

def synonym_variants(query: str, max_per_term: int = 2, max_variants: int = 3) -> List[str]:
    """Generates synonymous queries using WordNet."""
    toks = _normalize_tokens(query)
    content = [t for t in toks if t not in _STOP and len(t) > 2]
    variants: Set[str] = set()
    if not content:
        return []
    for term in content[:6]:
        syns = []
        for syn in wn.synsets(term):
            for lemma in syn.lemmas():
                w = lemma.name().replace("_", " ").lower()
                if w != term and w.isalpha() and len(w) > 2:
                    syns.append(w)
        counts = Counter(syns)
        for alt, _ in counts.most_common(max_per_term):
            new_toks = [alt if t == term else t for t in content]
            variants.add(" ".join(new_toks))
            if len(variants) >= max_variants:
                break
        if len(variants) >= max_variants:
            break
    return list(variants)

def generate_multi_queries(query: str, max_variants: int = 5) -> List[str]:
    """Combines original, condensed, and synonym queries."""
    q0 = query.strip()
    q1 = condense_query(q0)
    q_syns = synonym_variants(q0, max_per_term=2, max_variants=max(0, max_variants - 2))
    seen = []
    for q in [q0, q1] + q_syns:
        if q and q not in seen:
            seen.append(q)
    return seen

In [ ]:
# --- Retrieval System ---

def clear_and_create_collection():
    try:
        chroma_client.delete_collection(name="stats_books")  # Consistent name
        print("Cleared existing collection")
    except Exception as e:
        print(f"No existing collection to clear ({e})")
    global collection
    collection = chroma_client.get_or_create_collection(name="stats_books")  # Same name
    print("Created fresh collection")

def retrieve_relevant_chunks(query, top_k):
    """Basic retrieval using cosine similarity."""
    query_embedding = embedder.encode([query])[0]

    results = collection.query(
        query_embeddings=[query_embedding.tolist()],  
        n_results=min(top_k * 2, 15),  # Get extra candidates
        include=["documents", "distances"])
    
    documents = results.get("documents", [[]])[0]
    return documents[:top_k]

def retrieve_pool_with_multiquery(query: str, collection, embedder, k_per_query: int = 8, verbose: bool = True):
    """Retrieves candidates using multiple query variations."""
    multi_queries = generate_multi_queries(query, max_variants=5)
    if verbose:
        print("Transformed queries:")
        for i, mq in enumerate(multi_queries, 1):
            print(f"  {i}. {mq}")

    pool: Dict[str, Dict[str, Any]] = {}
    for mq in multi_queries:
        q_emb = embedder.encode([mq])[0].tolist()
        res = collection.query(
            query_embeddings=[q_emb],
            n_results=int(k_per_query),
            include=["documents", "metadatas", "distances", "ids"] # Ensure IDs are included
        )
        docs = res.get("documents", [[]])[0]
        metas = res.get("metadatas", [[]])[0]
        dists = res.get("distances", [[]])[0]
        ids   = res.get("ids", [[]])[0]

        for _id, d, m, dist in zip(ids, docs, metas, dists):
            if _id not in pool or dist < pool[_id]["distance"]:
                pool[_id] = {"id": _id, "doc": d, "meta": m, "distance": float(dist)}

    if verbose:
        print(f"\nCandidate pool size (deduped): {len(pool)}")
    return list(pool.values())

def rerank_candidates(query: str, candidates: list, top_k: int = 4, verbose: bool = True):
    """Reranks candidates using CrossEncoder."""
    if not candidates:
        if verbose:
            print("No candidates to rerank.")
        return []
    pairs = [(query, c["doc"]) for c in candidates]
    scores = reranker.predict(pairs)
    for c, s in zip(candidates, scores):
        c["rerank_score"] = float(s)
    candidates.sort(key=lambda x: x["rerank_score"], reverse=True)
    top = candidates[:top_k]

    if verbose:
        print(f"\nTop {top_k} after rerank:")
        for i, c in enumerate(top, 1):
            # Printing meta data logic kept from your code
            meta_str = ""
            if isinstance(c.get("meta"), dict):
                src = c["meta"].get("source") or c["meta"].get("file") or c["meta"].get("path")
                page = c["meta"].get("page") or c["meta"].get("page_number")
                meta_bits = []
                if src: meta_bits.append(f"src={src}")
                if page: meta_bits.append(f"page={page}")
                meta_str = " | " + ", ".join(meta_bits) if meta_bits else ""
            print(f"  {i}. score={c['rerank_score']:.4f}  dist={c['distance']:.4f}{meta_str}")
    return top

def retrieve_with_transform_and_rerank(query: str, collection, embedder,
                                       k_per_query: int = 8, k_final: int = 4, verbose: bool = True):
    """End-to-end retrieval pipeline."""
    pool = retrieve_pool_with_multiquery(query, collection, embedder,
                                         k_per_query=k_per_query, verbose=verbose)
    topk = rerank_candidates(query, pool, top_k=k_final, verbose=verbose)
    return topk

In [ ]:
# ===========================
# RESEARCH-BACKED RAG EVALUATION 
# Citations: Lewis et al. (2020), Karpukhin et al. (2020)
# ===========================

def calculate_semantic_precision_recall(query, retrieved_chunks, ground_truth_answer):
    """
    Research-backed semantic precision/recall for RAG evaluation.
    Based on: Lewis et al. (2020) - RAG paper, Karpukhin et al. (2020) - DPR paper
    """
    if not retrieved_chunks or not ground_truth_answer:
        return {'precision': 0, 'recall': 0, 'f1_score': 0}
    
    # Use your existing embedder
    global embedder
    
    # 1. Encode all content
    query_emb = embedder.encode(query)
    truth_emb = embedder.encode(ground_truth_answer)
    retrieved_embs = embedder.encode(retrieved_chunks)
    
    # 2. Calculate cosine similarities
    from sentence_transformers import util
    query_similarities = util.cos_sim(query_emb, retrieved_embs)[0]
    truth_similarities = util.cos_sim(truth_emb, retrieved_embs)[0]
    
    # 3. Research standard: relevance threshold 0.6
    relevance_threshold = 0.6
    
    # 4. Precision: How many retrieved chunks are relevant to query?
    relevant_chunks = (query_similarities > relevance_threshold).sum().item()
    precision = relevant_chunks / len(retrieved_chunks)
    
    # 5. Recall: Best chunk's similarity to ground truth
    recall = truth_similarities.max().item()
    
    # 6. F1 Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall, 
        'f1_score': f1,
        'avg_similarity': query_similarities.mean().item()
    }

def calculate_retrieval_metrics(questions, answers, top_k):
    """Calculate proper retrieval metrics for multiple questions"""
    
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    for i, (question, answer) in enumerate(zip(questions, answers)):
        if i >= len(answers):
            break
            
        try:
            # Get retrieval results using your existing function
            retrieved_chunks = retrieve_relevant_chunks(question, top_k)
            
            # Calculate semantic metrics
            metrics = calculate_semantic_precision_recall(question, retrieved_chunks, answer)
            
            precision_scores.append(metrics['precision'])
            recall_scores.append(metrics['recall'])
            f1_scores.append(metrics['f1_score'])
            
        except Exception as e:
            print(f"Error processing question {i}: {e}")
            precision_scores.append(0)
            recall_scores.append(0)
            f1_scores.append(0)
    
    return {
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'f1_score': np.mean(f1_scores),
        'precision_std': np.std(precision_scores),
        'recall_std': np.std(recall_scores),
        'f1_std': np.std(f1_scores)
    }

def clean_csv_data(csv_path):
    """Cleans the dataset for processing."""
    qa_df = pd.read_csv(csv_path)
    qa_df = qa_df.dropna(subset=['Question', 'Human_Answer'])

    questions = []
    answers = []
    
    for i, row in qa_df.iterrows():
        q = str(row['Question']).strip()
        a = str(row['Human_Answer']).strip()
        
        if q and a and q != 'nan' and a != 'nan' and len(q) > 5:
            questions.append(q)
            answers.append(a)
    
    print(f"Cleaned data: {len(questions)} valid Q&A pairs")
    return questions, answers

In [ ]:
def add_chunks_to_chromadb(chunks, batch_size=1000):
    """Helper to add chunks in batches."""
    texts = [chunk.page_content for chunk in chunks]
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        embeddings = embedder.encode(batch_texts)
        
        batch_ids = [f"chunk_{j}" for j in range(i, i+len(batch_texts))]
        batch_metadata = [{"source": f"chunk_{j}"} for j in range(i, i+len(batch_texts))]
        
        collection.add(documents=batch_texts, embeddings=embeddings.tolist(), metadatas=batch_metadata, ids=batch_ids)
        print(f"✅ Added batch {i//batch_size + 1}/{math.ceil(len(texts)/batch_size)}")
    
    print("All chunks successfully added to ChromaDB")
    return collection

def test_chunk_params_semantic(dataset, documents, chunk_sizes, chunk_overlaps, top_k):
    """Test chunking parameters using research-backed semantic evaluation"""
    
    questions, answers = clean_csv_data(dataset)
    results = []
    
    for chunk_size in chunk_sizes:
        for chunk_overlap in chunk_overlaps:
            print(f"\n🔧 Testing: chunk_size={chunk_size}, overlap={chunk_overlap}")
            
            if chunk_overlap >= chunk_size:  
                print(f"❌ Skipping: overlap >= chunk_size")
                continue

            # Setup collection with new parameters
            clear_and_create_collection()
            chunks = split_text(documents, chunk_size=chunk_size, overlap=chunk_overlap)
            collection = add_chunks_to_chromadb(chunks)
            
            # Calculate semantic metrics (use first 20 questions for speed)
            test_questions = questions[:20]
            test_answers = answers[:20]
            
            print(f"📊 Evaluating on {len(test_questions)} questions...")
            metrics = calculate_retrieval_metrics(test_questions, test_answers, top_k)
            
            results.append({
                'chunk_size': chunk_size,
                'chunk_overlap': chunk_overlap,
                'precision': metrics['precision'],
                'recall': metrics['recall'],
                'f1_score': metrics['f1_score'],
                'precision_std': metrics['precision_std'],
                'recall_std': metrics['recall_std'],
                'f1_std': metrics['f1_std']
            })
            
            print(f"✅ Results: P={metrics['precision']:.3f}±{metrics['precision_std']:.3f}, R={metrics['recall']:.3f}±{metrics['recall_std']:.3f}, F1={metrics['f1_score']:.3f}±{metrics['f1_std']:.3f}")
    
    results_df = pd.DataFrame(results)
    return results_df

def optimize_top_k_semantic(dataset, documents, best_chunk_size, best_overlap, top_k_values):
    """Optimize top-k using semantic evaluation"""
    
    questions, answers = clean_csv_data(dataset)
    
    # Setup collection with best parameters
    clear_and_create_collection()
    chunks = split_text(documents, chunk_size=best_chunk_size, overlap=best_overlap)
    add_chunks_to_chromadb(chunks)

    results = []

    for top_k in top_k_values:
        print(f"🔍 Testing top_k={top_k}")
        
        # Use first 20 questions for evaluation
        test_questions = questions[:20]
        test_answers = answers[:20]
        
        metrics = calculate_retrieval_metrics(test_questions, test_answers, top_k)
        
        results.append({
            'top_k': top_k, 
            'precision': metrics['precision'],
            'recall': metrics['recall'],
            'f1_score': metrics['f1_score']
        })
        
        print(f"   F1: {metrics['f1_score']:.3f}")

    results_df = pd.DataFrame(results)
    best_config = results_df.loc[results_df['f1_score'].idxmax()]

    return results_df, best_config

In [ ]:
def llm_baseline_answer(query, model_name):
    """Generates an answer using only parametric knowledge."""
    response = ollama_client.generate(
        model=model_name,
        prompt=f"""You are a statistics expert. Answer the question step-by-step: 
        Question: {query}""",

        stream=False,
        options={
        'num_predict': 1000,    # Longer responses
        'temperature': 0.2,    # More consistent 
        'top_p': 0.9  
    }
    )
    return response['response']

def llm_rag_answer_simplified(query, model_name, top_k):
    """Generates answer using RAG context."""
    
    # Get relevant chunks using simple retrieval
    relevant_chunks = retrieve_relevant_chunks(query, top_k)
    
    # Minimal context approach
    context = relevant_chunks[:300]  # Limit context length
    
    # Simple, direct prompt (no complex instructions)
    prompt = f"""You are a statistics expert. Answer the question step-by-step: 
    Statistical question: {query}

Reference material (use only if directly relevant): 
{context}

Answer:"""
    
    response = ollama_client.generate(
        model=model_name,
        prompt=prompt,
        stream=False,
        options={
            'num_predict': 1000,
            'temperature': 0.2,
            'top_p': 0.9
        }
    )
    
    return response['response']

In [ ]:
# Load Main Dataset
df2 = pd.read_csv(dataset_main)

# Initialize Results Dataframe
df = pd.DataFrame({
    'Question Topic' : df2["Question Type"],
    'Question': df2["Question"],
    'Original_Answer': df2["Human_Answer"],

    'Model1_Baseline': None,
    'Model2_Baseline': None,
    'Model3_Baseline': None,

    'Model1_RAG': None,
    'Model2_RAG': None,
    'Model3_RAG': None,

})

# Models
model1 = "llama3.2:latest"
model2 = "deepseek-r1:8b"
model3 = "qwen2-math:7b"

# Use optimized top_k from previous steps (hardcoded here if optimization skipped)
optimal_top_k = 3 # based on your results

# Save parameters for reference
import json
optimization_params = {
    'top_k': optimal_top_k
}

start_index = 0

print(f"🚀 Starting Main Experiment Loop...")
print(f"   Models: {model1}, {model2}, {model3}")

for i in range(start_index, len(df)):
    Question = df.at[i, 'Question']
    Original_Answer = df.at[i, 'Original_Answer']

    print(f"Question {i+1}/{len(df)}:")

    # Generate responses using optimized parameters
    llm_baseline_model1 = llm_baseline_answer(Question, model1)
    llm_rag_model1 = llm_rag_answer_simplified(Question, model1, optimal_top_k)  

    llm_baseline_model2 = llm_baseline_answer(Question, model2)
    llm_rag_model2 = llm_rag_answer_simplified(Question, model2, optimal_top_k)  
    
    llm_baseline_model3 = llm_baseline_answer(Question, model3)
    llm_rag_model3 = llm_rag_answer_simplified(Question, model3, optimal_top_k)  

    # Store results
    df.at[i, 'Question'] = Question
    df.at[i, 'Original_Answer'] = Original_Answer

    df.at[i, 'Model1_Baseline'] = llm_baseline_model1
    df.at[i, 'Model2_Baseline'] = llm_baseline_model2
    df.at[i, 'Model3_Baseline'] = llm_baseline_model3

    df.at[i, 'Model1_RAG'] = llm_rag_model1
    df.at[i, 'Model2_RAG'] = llm_rag_model2
    df.at[i, 'Model3_RAG'] = llm_rag_model3

    print('-' * 40)

    # Save intermediate results
    df.to_csv('results.csv', index=False)
    
print("✅ Experiment completed!")

In [ ]:
def evaluate_answer(question, llm_answer, original_answer, judge_llm):
    """
    Uses an LLM to score the answer based on Correctness, Explanation, and Understanding.
    """
    prompt = f"""You are a statistics evaluator. Score the answer on the basis of its correctness, explainability and understanding. Be precise and score it between 0 and 5, also allowing for half-point increments (e.g., 2.5, 3.5).

The rubrics is provided below, be very precise about your evaluations.
RUBRIC:
1. **CORRECTNESS**:
    5: Correct answer AND correct method
    4: Correct method, minor computational error
    3: Partially correct approach
    2: Wrong approach but shows some knowledge
    1: Completely wrong or no attempt

 2. **EXPLANATION**:
    5: Clear, complete, step-by-step explanation
    4: Generally clear with minor gaps
    3: Adequate explanation with some confusion
    2: Unclear but attempted explanation
    1: No explanation or incomprehensible

 3. **UNDERSTANDING**:
    5: Shows deep understanding (discusses assumptions, limitations, context)
    4: Shows good understanding with minor gaps
    3: Shows basic understanding
    2: Shows minimal understanding
    1: No conceptual understanding evident

---

### QUESTION:
{question}

### ORIGINAL ANSWER:
{original_answer}

### LLM-GENERATED ANSWER:
{llm_answer}

---

IMPORTANT: Reply with ONLY this JSON format, nothing else:
{{{"correctness": X, "explanation": X, "understanding": X}}}

JSON OUTPUT:"""
    
    response = ollama_client.generate(
        model=judge_llm,
        prompt=prompt,
        stream=False,
        options={
            "temperature": 0.1,  # Low temperature for consistency
            "num_predict": 50,   # Limit output length
            'stop': ['\n\n', 'Human:']        # Stop after closing bracket
        }
    )

    raw_text = response['response']
    return raw_text

def score_break(score_text):
    """Simple fix for parsing scores"""
    if not score_text:
        return 1, 1, 1
    
    # Just extract the numbers, ignore JSON formatting
    import re
    numbers = re.findall(r'\d+', str(score_text))
    
    if len(numbers) >= 3:
        return int(numbers[0]), int(numbers[1]), int(numbers[2])
    else:
        return 1, 1, 1

In [ ]:
judge_llms = {
    'Prometheus': 'vicgalle/prometheus-7b-v2.0:latest',
    # 'Llama32': 'llama3.2:latest',
    # 'Mistral': 'mistral:latest',
    # 'Gemma': 'gemma3n:latest'
}

start_index = 0 

print("⚖️ Starting Evaluation Phase...")

for i in range(start_index, len(df)):
    Question = df.at[i, 'Question']
    Original_Answer = df.at[i, 'Original_Answer']
    
    print(f"Row {i+1}/{len(df)}")
    
    for judge_name, judge_llm in judge_llms.items():
        print(f"  Judge: {judge_name}")
        
        for model in ['Model1', 'Model2', 'Model3']:
            for condition in ['Baseline', 'RAG']:
                response = df.at[i, f"{model}_{condition}"]
                
                try:
                    # Add debugging
                    # print(f"    Evaluating {model}_{condition}...")
                    score = evaluate_answer(Question, Original_Answer, response, judge_llm)
                    
                    if not score or score.strip() == "":
                        print(f"    Empty score for {model}_{condition}, skipping...")
                        continue
                    
                    Correct, Explain, Understand = score_break(score)
                    
                    df.at[i, f"{model}_{condition}_{judge_name}_Correctness"] = Correct
                    df.at[i, f"{model}_{condition}_{judge_name}_Explanation"] = Explain
                    df.at[i, f"{model}_{condition}_{judge_name}_Understanding"] = Understand
                    
                except Exception as e:
                    print(f"    ERROR with {judge_name} on {model}_{condition}: {e}")
                    # Set default values on error
                    df.at[i, f"{model}_{condition}_{judge_name}_Correctness"] = 1
                    df.at[i, f"{model}_{condition}_{judge_name}_Explanation"] = 1
                    df.at[i, f"{model}_{condition}_{judge_name}_Understanding"] = 1
    
    # Save after each row to prevent data loss
    if (i + 1) % 5 == 0:
        df.to_csv('results_progress.csv', index=False)
        print(f"  Saved progress at row {i+1}")

    df.to_csv('results.csv', index=False)
    
print("✅ Evaluation Complete.")

In [ ]:
# --- Multi-Judge Consensus ---

judges = ['Prometheus', 'Llama32', 'Mistral', 'Gemma']

print("=== EVALUATE BEST JUDGE ===\n")

# Calculate consensus scores
for model in ['Model1', 'Model2', 'Model3']:
    for condition in ['Baseline', 'RAG']:
        for score_type in ['Correctness', 'Explanation', 'Understanding']:
            judge_cols = [f'{model}_{condition}_{judge}_{score_type}' for judge in judges]
            existing_cols = [col for col in judge_cols if col in df.columns]
            
            if len(existing_cols) >= 2:
                consensus_col = f'{model}_{condition}_Consensus_{score_type}'
                df[consensus_col] = df[existing_cols].mean(axis=1)

print("✅ Consensus calculated")

# Calculate correlations
judge_correlations = {}
for judge in judges:
    judge_scores = []
    consensus_scores = []
    
    for model in ['Model1', 'Model2', 'Model3']:
        for condition in ['Baseline', 'RAG']:
            for score_type in ['Correctness', 'Explanation', 'Understanding']:
                judge_col = f'{model}_{condition}_{judge}_{score_type}'
                consensus_col = f'{model}_{condition}_Consensus_{score_type}'
                
                if judge_col in df.columns and consensus_col in df.columns:
                    j_scores = df[judge_col].dropna().tolist()
                    c_scores = df[consensus_col].dropna().tolist()
                    
                    min_len = min(len(j_scores), len(c_scores))
                    judge_scores.extend(j_scores[:min_len])
                    consensus_scores.extend(c_scores[:min_len])
    
    if len(judge_scores) > 5:
        correlation, p_value = pearsonr(judge_scores, consensus_scores)
        judge_correlations[judge] = correlation
        print(f"{judge}: correlation = {correlation:.4f}")

best_judge = max(judge_correlations, key=judge_correlations.get)
print(f"\n🏆 BEST JUDGE: {best_judge}")

In [ ]:
# --- Analysis & Visualization ---

# Clean up column names based on selected judge
selected_llm_judge = "Prometheus"
score_col = list()
for col in df.columns:
    if selected_llm_judge in col:
        score_col.append(col)

QA_col = ["Question Topic","Question","Original_Answer",'Model1_Baseline','Model2_Baseline', 'Model3_Baseline', 'Model1_RAG', 'Model2_RAG', 'Model3_RAG']
final_col = QA_col + score_col

df1 = df[final_col]
column_names = df1.columns
new_column_names = [col.replace(f"_{selected_llm_judge}", '') for col in column_names]
df1.columns = new_column_names

# Calculate Weighted Totals
df1["Model1_Total_Baseline"] = (0.5 * df1["Model1_Baseline_Correctness"]) + (0.3 * df1["Model1_Baseline_Explanation"]) + (0.2 * df1["Model1_Baseline_Understanding"])
df1["Model2_Total_Baseline"] = (0.5 * df1["Model2_Baseline_Correctness"]) + (0.3 * df1["Model2_Baseline_Explanation"]) + (0.2 * df1["Model2_Baseline_Understanding"])
df1["Model3_Total_Baseline"] = (0.5 * df1["Model3_Baseline_Correctness"]) + (0.3 * df1["Model3_Baseline_Explanation"]) + (0.2 * df1["Model3_Baseline_Understanding"])

df1["Model1_Total_RAG"] = (0.5 * df1["Model1_RAG_Correctness"]) + (0.3 * df1["Model1_RAG_Explanation"]) + (0.2 * df1["Model1_RAG_Understanding"])
df1["Model2_Total_RAG"] = (0.5 * df1["Model2_RAG_Correctness"]) + (0.3 * df1["Model2_RAG_Explanation"]) + (0.2 * df1["Model2_RAG_Understanding"])
df1["Model3_Total_RAG"] = (0.5 * df1["Model3_RAG_Correctness"]) + (0.3 * df1["Model3_RAG_Explanation"]) + (0.2 * df1["Model3_RAG_Understanding"])

# Plotting
models = ['Model1', 'Model2', 'Model3']
fig, ax = plt.subplots(figsize=(8, 5))

baseline = [df1[f'{m}_Total_Baseline'].mean() for m in models]
rag = [df1[f'{m}_Total_RAG'].mean() for m in models]

x = range(len(models))
width = 0.35

ax.bar([i-width/2 for i in x], baseline, width, label='Baseline', color='blue', alpha=0.7)
ax.bar([i+width/2 for i in x], rag, width, label='RAG', color='red', alpha=0.7)

ax.set_xlabel('Models')
ax.set_ylabel('Total Score')
ax.set_title('Overall Performance: RAG vs Baseline')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
plt.show()

In [ ]:
# --- Component-Level Analysis ---

components = ['Correctness', 'Explanation', 'Understanding']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
width = 0.35

for i, comp in enumerate(components):
    baseline = [df1[f'{m}_Baseline_{comp}'].mean() for m in models]
    rag = [df1[f'{m}_RAG_{comp}'].mean() for m in models]
    
    x = range(len(models))
    axes[i].bar([j-width/2 for j in x], baseline, width, 
               label='Baseline', color='blue', alpha=0.7)
    axes[i].bar([j+width/2 for j in x], rag, width, 
               label='RAG', color='red', alpha=0.7)
    
    axes[i].set_title(comp, fontweight='bold')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(models)
    axes[i].grid(axis='y', alpha=0.3)
    if i == 0:
        axes[i].legend()

plt.suptitle('Performance Breakdown by Evaluation Dimensions', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# --- Performance by Statistical Domain ---

question_types = ['Probability', 'Hypothesis Testing', 'Regression and Correlation']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, qtype in enumerate(question_types):
    subset = df1[df1['Question Topic'] == qtype]
    baseline = [subset[f'{m}_Total_Baseline'].mean() for m in models]
    rag = [subset[f'{m}_Total_RAG'].mean() for m in models]
    
    x = range(len(models))
    axes[i].bar([j-width/2 for j in x], baseline, width, 
               label='Baseline', color='blue', alpha=0.7)
    axes[i].bar([j+width/2 for j in x], rag, width, 
               label='RAG', color='red', alpha=0.7)
    
    axes[i].set_title(qtype, fontsize=11, fontweight='bold')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(models, fontsize=9)
    axes[i].grid(axis='y', alpha=0.3)
    if i == 0:
        axes[i].legend()

plt.suptitle('Performance by Statistical Domain', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Statistics ---

print("=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

for model in models:
    print(f"\n🤖 {model}:")
    
    baseline_mean = df1[f'{model}_Total_Baseline'].mean()
    rag_mean = df1[f'{model}_Total_RAG'].mean()
    diff = rag_mean - baseline_mean
    pct_change = (diff / baseline_mean) * 100
    
    print(f"   Baseline:  {baseline_mean:.3f}")
    print(f"   RAG:       {rag_mean:.3f}")
    print(f"   Change:    {diff:+.3f} ({pct_change:+.1f}%)")

print("\n" + "=" * 70)